In [4]:
# Library Import

import os 
import importlib.util

if importlib.util.find_spec("pandas") is None:	
	os.system("pip install pandas")

In [5]:
# pandas Config

import pandas as pd

pd.options.mode.use_inf_as_na = True
pd.set_option("display.max_rows", 10)
pd.set_option("display.expand_frame_repr", True)
pd.set_option('display.width', 1000)
pd.options.display.max_seq_items = 200000
pd.options.display.max_rows = 10

In [6]:
# Parameters

from ipython_secrets import get_secret

Host = get_secret('SECRETS_HOST') 
Username = get_secret('SECRETS_USERNAME') 
Password = get_secret('SECRETS_PASSWORD') 
ValidProjectCategories = ["'Customer Delivery Projects'"]

In [7]:
# Functions

import base64
import pandas as pd
import re 
import requests
from functools import reduce
    
def _ExpandColumn(self:pd.DataFrame, colName:str, columnsToExpand = [], prefix:str = "Prefix", sentenceCase:bool = True) -> pd.DataFrame:
    if (prefix == "Prefix"):
        prefix = colName + " "
        expandedCols = self[colName].apply(lambda x: pd.Series(x).add_prefix(prefix))
        columnsToExpand = [prefix + c for c in columnsToExpand]
    else:
        expandedCols = self[colName].apply(lambda x: pd.Series(x))
    
    if len(columnsToExpand) > 0:        
        expandedCols = expandedCols[columnsToExpand]
    
    if sentenceCase:
        expandedCols.columns = [fnSentenceCase(c) for c in expandedCols.columns] 

    return pd.concat([self.drop(colName, axis=1), expandedCols], axis=1)

pd.DataFrame.expand = _ExpandColumn

def fnSentenceCase(s):
    s = (' '.join(dict.fromkeys(s.split())))  # remove duplicate words
    s = s.replace("0", "") # remove "0" 
    s = s.strip()
    return ' '.join([x.capitalize() for x in re.sub(r"([A-Z])", r" \1", s).split()]) # sentence case

def _SentenceCaseColumns(self:pd.DataFrame) -> pd.DataFrame: 
    self.columns = [fnSentenceCase(c) for c in self.columns] 
    return self

pd.DataFrame.sentence_case_columns = _SentenceCaseColumns

def fnGetDefaultHeaders():
    return {
        "content-type": "application/json",
        "authorization": "Basic " + base64.b64encode((Username + ":" + Password).encode()).decode(),
        "retry-after": "120"
    }

def fnSearch(jql, fields = None, expand = None):
    def ApiCall(startAt) :
        url = "/rest/api/latest/search"
        headers = fnGetDefaultHeaders()
        defaultContents = {
            "startAt": startAt,
            "maxResults": "2",
            "jql": jql
        }
        if fields is not None:
            defaultContents["fields"] = fields.tolist()
        if expand is not None and expand != "":
            defaultContents["expand"] = expand        

        response = requests.post(Host + url, headers = headers, json = defaultContents)
        return response.json()
    values = fnAPI(ApiCall)
    if len(values.index) > 1:
        return values
    else:
        None

def fnGetIssueTypeFields(IssueTypes) -> pd.DataFrame:
    def ApiCall(startAt) :
        url = "rest/api/latest/issue/createmeta"
        headers = fnGetDefaultHeaders()
        params = {
            "expand": "projects.issuetypes.fields",
            "projectKeys": ','.join(fnGetValidProjectKeys()["key"].values),
            "issuetypeNames": ','.join(IssueTypes).replace("'", "")
        }
        response = requests.get(Host + url, headers = headers, params = params)
        return response.json()
    
    df = fnAPI(ApiCall)
    df = df.drop(["expand"], axis=1)
    df = df.explode("projects")
    df = df.expand("projects", [], None, False)
    df = df[["issuetypes"]]
    df = df.explode("issuetypes")
    df = df.expand("issuetypes", [], None, False)
    df = df[["fields"]]
    df = df.expand("fields", [], None, False)
    df = df.loc[:,~df.columns.duplicated()]

    values = []
    for x in df.columns:        
        try:            
            valid = pd.DataFrame( df[~df[x].isnull()] )[[x]].iloc[0].get(0)
            values.append({
                "fieldId": valid['key'],
                "name": valid['name'],
                "schema_type": valid['schema']['type'],
                "required": valid['required']
            })
        except:
            display(x)
            pass
    values.append({ "fieldId": 'status', "name": 'Status', "schema_type": 'string', "required": True })
    values.append({ "fieldId": 'created', "name": 'Created', "schema_type": 'date', "required": True })
    values.append({ "fieldId": 'updated', "name": 'Updated', "schema_type": 'date', "required": True })
    values.append({ "fieldId": 'resolution', "name": 'Resolution', "schema_type": 'string', "required": True })
    values.append({ "fieldId": 'resolutiondate', "name": 'Resolution Date', "schema_type": 'date', "required": False })
    values.append({ "fieldId": 'lastViewed', "name": 'Last Viewed', "schema_type": 'date', "required": True })
    values.append({ "fieldId": 'id', "name": 'Id', "schema_type": 'number', "required": True })
    values.append({ "fieldId": 'key', "name": 'Key', "schema_type": 'string', "required": True })
    df = pd.DataFrame(values)
    df = df.drop_duplicates().sort_values("fieldId")
    return df 

def fnGetValidProjectKeys() -> pd.DataFrame:
    def ApiCall(startAt) :
        url = "/rest/api/latest/project"
        headers = fnGetDefaultHeaders()
        params = { }
        response = requests.get(Host + url, headers = headers, params = params)
        return response.json()
    df = fnAPI(ApiCall)
    df = df.expand("projectCategory")
    if 'ValidProjectCategories' in globals() and len(ValidProjectCategories) > 0:
        df = df.loc[df['Project Category Name'].isin(ValidProjectCategories) | ("'" + df['Project Category Name'] + "'").isin(ValidProjectCategories)]
    return df[["key"]]

def fnAPI(webRequestDelegate, startAt = 0) -> pd.DataFrame:
    def flatten_reduce_lambda(frm):
        try:
            return list(reduce(lambda x, y: x + y, frm, []))         
        except:
            return list(reduce(lambda x, y: x + y, [frm], [])) 
    def innerGetResults(webRequestDelegate, startAt = 0):
        results = webRequestDelegate(startAt)
        if isinstance(results, dict) and "total" in results and "maxResults" in results:
            if startAt + results["maxResults"] < results["total"]:
                return [results] + innerGetResults(webRequestDelegate, startAt + results["maxResults"])
            else:
                return [results]
        else:
            return [results]
    Source = flatten_reduce_lambda(innerGetResults(webRequestDelegate, startAt))
    df = pd.DataFrame(Source)
    return df

In [8]:
# Projects

from IPython.display import display
import requests
from pandas.core.dtypes.dtypes import DatetimeTZDtype

def ApiCall(startAt) :
    url = "/rest/api/latest/project"
    headers = fnGetDefaultHeaders()
    params = {
        "startAt": startAt,
        "expand": "projectKeys"
    }
    response = requests.get(Host + url, headers = headers, params = params)
    return response.json()

df = fnAPI(ApiCall)  
df = df.drop(["avatarUrls", "expand", "self", "entityId", "uuid", "projectKeys", "properties"], axis=1).sentence_case_columns()
df = df.expand("Project Category", ["id", "name"])
df = df.expand("Archived By", ["accountId", "displayName"])
df["Is Valid"] = (
    df['Project Category Name'].isin(ValidProjectCategories) | ("'" + df['Project Category Name'] + "'").isin(ValidProjectCategories) if 'ValidProjectCategories' in globals() and len(ValidProjectCategories) > 0 else True
)

df = df.sentence_case_columns().convert_dtypes().infer_objects()
df["Id"] = df["Id"].astype('Int64')
df["Project Category Id"] = df["Project Category Id"].astype('Int64')
df["Archived Date"] = df["Archived Date"].astype(DatetimeTZDtype("ns", "+10:00"))

display(df.dtypes)
display(df)

Id                                   Int64
Key                         string[python]
Name                        string[python]
Project Type Key            string[python]
Simplified                         boolean
                                 ...      
Project Category Id                  Int64
Project Category Name       string[python]
Archived By Account Id      string[python]
Archived By Display Name    string[python]
Is Valid                           boolean
Length: 14, dtype: object

,Id,Key,Name,Project Type Key,Simplified,Style,Is Private,Archived,Archived Date,Project Category Id,Project Category Name,Archived By Account Id,Archived By Display Name,Is Valid
0,13098,AHPRA,AHPRA,service_desk,False,classic,True,<NA>,NaT,10106,Customer Delivery Projects (Atlassian),<NA>,<NA>,False
1,13177,AAS,AHPRA Application Support,service_desk,False,classic,True,<NA>,NaT,10106,Customer Delivery Projects (Atlassian),<NA>,<NA>,False
2,13091,AWS,Amazon Web Services,business,False,classic,False,<NA>,NaT,10102,Internal Projects,<NA>,<NA>,False
3,13173,ANGT,Anglicare,service_desk,False,classic,True,<NA>,NaT,10106,Customer Delivery Projects (Atlassian),<NA>,<NA>,False
4,13101,PD,Archived Project Delivery,software,False,classic,False,<NA>,NaT,<NA>,<NA>,<NA>,<NA>,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,13043,PP,ZZ - PinPoint (Pest Monitoring),software,False,classic,False,True,2022-07-27 16:09:13.865000+10:00,10104,Product Development,62786daa0e2c490069023bf0,bibek behera,False
78,13005,SG738,ZZ - QGCIO Cybersecurity (SG738),software,False,classic,False,True,2022-07-27 16:09:27.771000+10:00,10101,Customer Delivery Projects,62786daa0e2c490069023bf0,bibek behera,True
79,13058,QTSIB,ZZ - Queensland Treasury - Strategic Informati...,software,False,classic,False,True,2022-07-27 16:09:33.217000+10:00,10101,Customer Delivery Projects,62786daa0e2c490069023bf0,bibek behera,True
80,13051,SE,ZZ - Softlink Education,software,False,classic,False,True,2022-07-27 16:09:38.499000+10:00,10101,Customer Delivery Projects,62786daa0e2c490069023bf0,bibek behera,True


In [9]:
# Project Keys

from IPython.display import display
import requests

def ApiCall(startAt) :
    url = "/rest/api/latest/project"
    headers = fnGetDefaultHeaders()
    params = {
        "startAt": startAt,
        "expand": "projectKeys"
    }
    response = requests.get(Host + url, headers = headers, params = params)
    return response.json()

df = fnAPI(ApiCall).sentence_case_columns()

df = df[["Id", "Key", "Project Keys"]]
df = df.explode("Project Keys")

df = df.sentence_case_columns().convert_dtypes().infer_objects()
df["Id"] = df["Id"].astype('Int64')

display(df.dtypes)
display(df)

Id                       Int64
Key             string[python]
Project Keys    string[python]
dtype: object

,Id,Key,Project Keys
0,13098,AHPRA,AHPRA
1,13177,AAS,AAS
2,13091,AWS,AWS
3,13173,ANGT,ANGT
3,13173,ANGT,A1
...,...,...,...
78,13005,SG738,QC
79,13058,QTSIB,QTSIB
80,13051,SE,SE
80,13051,SE,STIH


In [10]:
# Project Properties

from IPython.display import display
import requests

def ApiCall(startAt) :
    url = "/rest/api/latest/project"
    headers = fnGetDefaultHeaders()
    params = {
        "startAt": startAt,
        "expand": "projectKeys"
    }
    response = requests.get(Host + url, headers = headers, params = params)
    return response.json()

df = fnAPI(ApiCall).sentence_case_columns()

df = df[["Id", "Key", "Properties"]]
df = df.explode("Properties")

df = df[~df["Properties"].isna()]
if df is None or len(df.index) == 0:	
    display("No results")
else:			
	df = df.sentence_case_columns().convert_dtypes().infer_objects()
	df["Id"] = df["Id"].astype('Int64')

	display(df.dtypes)
	display(df)

'No results'

In [12]:
# Project Categories

from IPython.display import display
import requests

def ApiCall(startAt) :
    url = "/rest/api/latest/projectCategory"
    headers = fnGetDefaultHeaders()
    params = {
        "startAt": startAt
    }
    response = requests.get(Host + url, headers = headers, params = params)
    return response.json()

df = fnAPI(ApiCall)  
df = df.drop(["self"], axis=1)

df = df.sentence_case_columns().convert_dtypes().infer_objects()
df["Id"] = df["Id"].astype('Int64')

list_row = [0, "No Category", "No Category"]
df.loc[len(df)] = list_row
df = df.sort_values('Id')

display(df.dtypes)
display(df)

Id              Int64
Description    object
Name           object
dtype: object

,Id,Description,Name
9,0,No Category,No Category
7,10000,,Software Development
8,10100,,Task Tracking
0,10101,A project set up to manage deliverables for a ...,Customer Delivery Projects
2,10102,Internal Strategenics projects,Internal Projects
6,10103,A project to support internal Strategenics Res...,Research & Development Projects
5,10104,A project for a product developed by Strategenics,Product Development
4,10105,Projects for ongoing managed services,Managed Services
1,10106,Atlassian Customer Delivery Projects,Customer Delivery Projects (Atlassian)
3,10107,,Internal Projects (Atlassian)


In [ ]:
# Issue Types

from IPython.display import display
import requests

def ApiCall(startAt) :
    url = "/rest/api/latest/issuetype"
    headers = fnGetDefaultHeaders()
    params = {
        "startAt": startAt
    }
    response = requests.get(Host + url, headers = headers, params = params)
    return response.json()

df = fnAPI(ApiCall)  
df = df.drop(["self", "scope"], axis=1)
df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)
df["Id"] = df["Id"].astype('Int64')

df = df.sort_values('Name').reset_index(drop=True)

display(df.dtypes)
display(df)

Id                            Int64
Description          string[python]
Icon Url             string[python]
Name                 string[python]
Untranslated Name    string[python]
Subtask                     boolean
Avatar Id                     Int64
Hierarchy Level               Int64
dtype: object

,Id,Description,Icon Url,Name,Untranslated Name,Subtask,Avatar Id,Hierarchy Level
0,10104,For new system accounts or passwords. Created ...,https://strategenics.atlassian.net/servicedesk...,Access,Access,False,<NA>,0
1,10984,Have a question? Submit it here.,https://strategenics.atlassian.net/rest/api/2/...,Ask a question,Ask a question,False,13390,0
2,1,A problem which impairs or prevents the functi...,https://strategenics.atlassian.net/rest/api/2/...,Bug,Bug,False,10303,0
3,10902,,https://strategenics.atlassian.net/rest/api/2/...,Bug*,Bug*,True,10303,-1
4,10102,For system upgrades or alterations. Created by...,https://strategenics.atlassian.net/servicedesk...,Change,Change,False,<NA>,0
5,10500,,https://strategenics.atlassian.net/rest/api/2/...,Change - Gratis,Change - Gratis,False,10309,0
6,10501,,https://strategenics.atlassian.net/rest/api/2/...,Change - New Feature,Change - New Feature,False,10314,0
7,10978,Send a request to make changes to your informa...,https://strategenics.atlassian.net/rest/api/2/...,Change request,Change request,False,13366,0
8,10905,,https://strategenics.atlassian.net/rest/api/2/...,Change*,Change*,True,11500,-1
9,10987,"A recipient of a good, service, product or an ...",https://strategenics.atlassian.net/rest/api/2/...,Customer,Customer,False,13400,0


In [ ]:
# Project Types

from IPython.display import display
import requests

def ApiCall(startAt) :
    url = "/rest/api/latest/project/type"
    headers = fnGetDefaultHeaders()
    params = {}
    response = requests.get(Host + url, headers = headers, params = params)
    return response.json()

df = fnAPI(ApiCall)  
df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)
df = df.sort_values('Key').reset_index(drop=True)

display(df.dtypes)
display(df)

Key                     string[python]
Formatted Key           string[python]
Description I18n Key    string[python]
Icon                    string[python]
Color                   string[python]
dtype: object

,Key,Formatted Key,Description I18n Key,Icon,Color
3,business,Business,jira.project.type.business.description,PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0iaXNvLT...,#1D8832
0,product_discovery,Product Discovery,jira.project.type.polaris.description,PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0iaXNvLT...,#F5A623
2,service_desk,Service Desk,jira.project.type.servicedesk.description.jsm,PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0iaXNvLT...,#67AB49
1,software,Software,jira.project.type.software.description,PD94bWwgdmVyc2lvbj0iMS4wIiBlbmNvZGluZz0iaXNvLT...,#F5A623


In [ ]:
# Resolutions

from IPython.display import display
import requests

def ApiCall(startAt) :
    url = "/rest/api/latest/resolution"
    headers = fnGetDefaultHeaders()
    params = {}
    response = requests.get(Host + url, headers = headers, params = params)
    return response.json()

df = fnAPI(ApiCall)  
df = df.drop(["self"], axis=1)
df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)
df["Id"] = df["Id"].astype('Int64')
df = df.sort_values('Name').reset_index(drop=True)

display(df.dtypes)
display(df)

Id                      Int64
Description    string[python]
Name           string[python]
dtype: object

,Id,Description,Name
2,10301,,Cancelled
7,10302,,Cannot Fix
5,5,"All attempts at reproducing this issue failed,...",Cannot Reproduce
11,10200,This issue was not approved.,Declined
0,10000,,Done
3,3,The problem is a duplicate of an existing issue.,Duplicate
1,1,A fix for this issue is checked into the tree ...,Fixed
12,10303,,Hardware failure
4,4,The problem is not completely described.,Incomplete
6,10201,The problem has a documented root cause and a ...,Known Error


In [ ]:
# Status

from IPython.display import display
import requests

def ApiCall(startAt) :
    url = "/rest/api/latest/status"
    headers = fnGetDefaultHeaders()
    params = {}
    response = requests.get(Host + url, headers = headers, params = params)
    return response.json()

df = fnAPI(ApiCall)  
df = df.drop(["self", "scope"], axis=1)
df = df.expand("statusCategory", ["id", "name"])

df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)
df["Id"] = df["Id"].astype('Int64')
df = df.sort_values('Name').reset_index(drop=True)

display(df.dtypes)
display(df)

Description             string[python]
Icon Url                string[python]
Name                    string[python]
Untranslated Name       string[python]
Id                               Int64
Status Category Id               Int64
Status Category Name    string[python]
dtype: object

,Description,Icon Url,Name,Untranslated Name,Id,Status Category Id,Status Category Name
0,,https://strategenics.atlassian.net/images/icon...,Assigned,Assigned,11955,2,To Do
1,This was auto-generated by Jira Service Manage...,https://strategenics.atlassian.net/images/icon...,Authorize,Authorize,11929,4,In Progress
2,This was auto-generated by Jira Service Desk d...,https://strategenics.atlassian.net/images/icon...,Awaiting implementation,Awaiting implementation,11847,4,In Progress
3,,https://strategenics.atlassian.net/images/icon...,Backlog,Backlog,10400,2,To Do
4,,https://strategenics.atlassian.net/images/icon...,Beta/UAT,Beta/UAT,11810,4,In Progress
5,This issue can't proceed due to a change in re...,https://strategenics.atlassian.net/images/icon...,Blocked by Change,Blocked by Change,4,2,To Do
6,"Source code has been committed, and JIRA is wa...",https://strategenics.atlassian.net/images/icon...,Building,Building,400,4,In Progress
7,This status is managed internally by JIRA Soft...,https://strategenics.atlassian.net/,"Built, Further Data Required","Built, Further Data Required",11308,4,In Progress
8,This status is managed internally by JIRA Soft...,https://strategenics.atlassian.net/,"Built, Ready For Review","Built, Ready For Review",11306,4,In Progress
9,This status is managed internally by JIRA Soft...,https://strategenics.atlassian.net/,"Built, Ready To Publish","Built, Ready To Publish",11309,4,In Progress


In [ ]:
# Status Categories

from IPython.display import display
import requests

def ApiCall(startAt) :
    url = "/rest/api/latest/statuscategory"
    headers = fnGetDefaultHeaders()
    params = {}
    response = requests.get(Host + url, headers = headers, params = params)
    return response.json()

df = fnAPI(ApiCall)  
df = df.drop(["self"], axis=1)

df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)
df["Id"] = df["Id"].astype('Int64')
df = df.sort_values('Id').reset_index(drop=True)

display(df.dtypes)
display(df)

Id                     Int64
Key           string[python]
Color Name    string[python]
Name          string[python]
dtype: object

,Id,Key,Color Name,Name
0,1,undefined,medium-gray,No Category
1,2,new,blue-gray,To Do
2,3,done,green,Done
3,4,indeterminate,yellow,In Progress


In [ ]:
# Users

from IPython.display import display
import requests
import pandas as pd

def ApiCall(startAt) :
    url = "/rest/api/latest/users/search"
    headers = fnGetDefaultHeaders()
    params = {
        "startAt": startAt,
        "maxResults": "1000"
    }
    response = requests.get(Host + url, headers = headers, params = params)
    return response.json()

df = pd.concat([fnAPI(ApiCall, x * 1000) for x in range(0, 10)])
df = df.drop(["self"], axis=1)

df = df.expand("avatarUrls", ["48x48"]).rename(columns={"Avatar Urls 48x48": "Avatar Url"})

df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)
df = df.sort_values('Display Name').reset_index(drop=True)

display(df.dtypes)
display(df)

Account Id       string[python]
Account Type     string[python]
Email Address    string[python]
Display Name     string[python]
Active                  boolean
Time Zone        string[python]
Locale           string[python]
Avatar Url       string[python]
dtype: object

,Account Id,Account Type,Email Address,Display Name,Active,Time Zone,Locale,Avatar Url
391,qm:0956ac3a-a022-4a10-a3c5-22e5fd6f75cf:5c80c3...,customer,<NA>,2014490606@qq.com,True,<NA>,<NA>,https://avatar-management--avatars.us-west-2.p...
919,6139d99124ba8b00701a07b1,atlassian,aws-support@strategenics.com.au,AWS-SUPPORT@strategenics.com.au,True,<NA>,<NA>,https://secure.gravatar.com/avatar/6fd0cc8a51c...
65,632113a7cd4a82cf977cafdb,atlassian,<NA>,Adrian Hill,True,<NA>,en_GB,https://secure.gravatar.com/avatar/939dc6b1cfe...
442,557057:fa71e034-c716-4a38-96eb-e3d8f5e106ca,atlassian,<NA>,Adven,False,<NA>,en_US,https://avatar-management--avatars.us-west-2.p...
932,5e5c7ada5a495e0c91a9a2a1,atlassian,<NA>,Agnieszka Klyz,True,<NA>,en_GB,https://secure.gravatar.com/avatar/a12986dd346...
...,...,...,...,...,...,...,...,...
111,qm:0956ac3a-a022-4a10-a3c5-22e5fd6f75cf:3e9f95...,customer,<NA>,wmalovnek@interactit.com.au,True,<NA>,<NA>,https://avatar-management--avatars.us-west-2.p...
266,qm:0956ac3a-a022-4a10-a3c5-22e5fd6f75cf:5c32c0...,customer,<NA>,yamiko.kambalametore@intertek.com,True,<NA>,<NA>,https://avatar-management--avatars.us-west-2.p...
27,qm:0956ac3a-a022-4a10-a3c5-22e5fd6f75cf:8c9ec0...,customer,<NA>,yasmineazzaoui2@gmail.com,True,<NA>,<NA>,https://avatar-management--avatars.us-west-2.p...
540,qm:0956ac3a-a022-4a10-a3c5-22e5fd6f75cf:101b82...,customer,<NA>,yourmail@gmail.com,True,<NA>,<NA>,https://avatar-management--avatars.us-west-2.p...


In [ ]:
# Priorities

from IPython.display import display
import requests

def ApiCall(startAt) :
    url = "/rest/api/latest/priority"
    headers = fnGetDefaultHeaders()
    params = {}
    response = requests.get(Host + url, headers = headers, params = params)
    return response.json()

df = fnAPI(ApiCall)  
df = df.drop(["self"], axis=1)

df = df.sentence_case_columns().convert_dtypes().infer_objects().reset_index(drop=True)
df["Id"] = df["Id"].astype('Int64')
df = df.sort_values('Id').reset_index(drop=True)

display(df.dtypes)
display(df)

Status Color    string[python]
Description     string[python]
Icon Url        string[python]
Name            string[python]
Id                       Int64
dtype: object

,Status Color,Description,Icon Url,Name,Id
0,#cc0000,to do,https://strategenics.atlassian.net/images/icon...,Critical,1
1,#ff0000,to do,https://strategenics.atlassian.net/images/icon...,High,2
2,#009900,to do,https://strategenics.atlassian.net/images/icon...,Medium,3
3,#006600,to do,https://strategenics.atlassian.net/images/icon...,Low,4
4,#003300,to do,https://strategenics.atlassian.net/images/icon...,Enhancement,5
